In [60]:
!pip install pandas matplotlib seaborn sklearn tensorflow
!pip install "numpy<2"


In [61]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [62]:
# Load the dataset
file_path = "./proccessedBookings.csv"
df = pd.read_csv(file_path)

# Display basic info
display(df.info())
display(df.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29990 entries, 0 to 29989
Data columns (total 47 columns):
 #   Column                                     Non-Null Count  Dtype  
---  ------                                     --------------  -----  
 0   Unnamed: 0.1                               29990 non-null  int64  
 1   Unnamed: 0                                 29509 non-null  float64
 2   name                                       29509 non-null  object 
 3   original_price                             29509 non-null  float64
 4   discounted_price                           29509 non-null  float64
 5   discount                                   29509 non-null  object 
 6   review_score                               29505 non-null  float64
 7   review_title                               28271 non-null  float64
 8   number_of_reviews                          29505 non-null  float64
 9   star_rating                                29202 non-null  float64
 10  is_apartment          

None

,Unnamed: 0.1,Unnamed: 0,name,original_price,discounted_price,discount,review_score,review_title,number_of_reviews,star_rating,...,"neighborhood_Midtown East, New York","neighborhood_Murray Hill, New York",neighborhood_New York,"neighborhood_NoMad, New York","neighborhood_SoHo, New York","neighborhood_Tribeca, New York","neighborhood_Union Square, New York","neighborhood_Upper East Side, New York","neighborhood_Upper West Side, New York","neighborhood_Washington Heights, New York"
0,0,0.0,Iroquois New York Times Square,292.0,218.0,True,8.5,4.0,2625.0,5.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,1.0,Four Points by Sheraton New York Downtown,130.0,111.0,True,7.8,3.0,1987.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,2.0,"The Mayfair Hotel Times Square, Ascend Hotel C...",141.0,141.0,False,8.6,5.0,1895.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,3.0,The Washington by LuxUrban,160.0,160.0,False,7.7,3.0,9442.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,4.0,Hampton Inn by Hilton New York Times Square,152.0,152.0,False,7.8,3.0,2432.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [63]:
# Feature Engineering
df['log_time_to_travel'] = np.log1p(df['time_to_travel'])  # Log transformation
df['long_stay'] = (df['length_of_stay'] > 3).astype(int)  # 1 if stay > 3 nights, else 0

In [64]:
print(df['discounted_price_per_night'].dtype)
print(df['discounted_price_per_night'])

float64
0        218.0
1        111.0
2        141.0
3        160.0
4        152.0
         ...  
29985    285.8
29986    244.4
29987    338.6
29988    317.4
29989    350.0
Name: discounted_price_per_night, Length: 29990, dtype: float64


In [65]:
# Define Features and Target
X = df[['log_time_to_travel', 'length_of_stay', 'long_stay']]  # Features
y = df['discounted_price_per_night']  # Target Variable

In [66]:
print(X.shape)
print(y.shape)
print(X.head())  # Check if X contains valid data
print(y.head())  # Check if y contains valid target values

(29990, 3)
(29990,)
   log_time_to_travel  length_of_stay  long_stay
0            0.693147             1.0          0
1            0.693147             1.0          0
2            0.693147             1.0          0
3            0.693147             1.0          0
4            0.693147             1.0          0
0    218.0
1    111.0
2    141.0
3    160.0
4    152.0
Name: discounted_price_per_night, dtype: float64


In [67]:
# Split Data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [68]:
# Normalize Data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [69]:
# Train Multiple Regression Models
models = {
    'Linear Regression': LinearRegression(),
    'Decision Tree': DecisionTreeRegressor(max_depth=5),
    'Gaussian Process': GaussianProcessRegressor(kernel=C(1.0) * RBF(1.0)),
    'Random Forest': RandomForestRegressor(n_estimators=100, max_depth=5),
    'Gradient Boosting': GradientBoostingRegressor(n_estimators=100, learning_rate=0.1),
    'SVR': SVR(kernel='rbf')
}

In [70]:
# Train and Evaluate Models
results = {}
for name, model in models.items():
    model.fit(X_train_scaled, y_train)
    y_pred = model.predict(X_test_scaled)
    results[name] = {
        'R2': r2_score(y_test, y_pred),
        'MAE': mean_absolute_error(y_test, y_pred),
        'MSE': mean_squared_error(y_test, y_pred),
        'RMSE': np.sqrt(mean_squared_error(y_test, y_pred))
    }

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').